In [2]:
import torch
from datasets import load_dataset
from datasets import load_from_disk


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:

# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # self.dataset = load_dataset('lansinuote/ChnSentiCorp', split=split)
        self.dataset = load_from_disk('./data/ChnSentiCorp')['%s' % split]
        print(self.dataset['text'][0])
        print(self.dataset['label'][0])
        # self.dataset = load_from_disk('./data/ChnSentiCorp')
        # print(self.dataset)
        # self.dataset = datasets.fetch_openml('./data/ChnSentiCorp', split=split)
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset['text'][i]
        label = self.dataset['label'][i]
        return text, label



In [4]:

# 打印数据
dataset = Dataset('train') #['train']

print(len(dataset))


for i in range(10):
    print('i: %s, label: %s %s' % (i, dataset[i][1],  dataset[i][0]))

选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
1
9600
i: 0, label: 1 选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
i: 1, label: 1 15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错
i: 2, label: 0 房间太小。其他的都一般。。。。。。。。。
i: 3, label: 0 1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.
i: 4, label: 1 今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。
i: 5, label: 0 机器背面似乎被撕了张什么标签，残胶还在。但是又看不出是什么标签不见了，该有的都在，怪
i: 6, label: 0 呵呵，虽然表皮看上去不错很精致，但是我还是能看得出来是盗的。但是里面的内容真的不错，我妈爱看，我自己也学着找一些穴位。
i: 7, label: 0 这本书实在是太烂了,以前听浙大的老师说这本书怎么怎么不对,哪些地方都是误导的还不相信,终于买了一本看一下,发现真是~~~无语,这种书都写得出来
i: 8, label: 1 地理位置佳，在市中心。酒店服务好、早餐品种丰富。我住的商务数码房电脑宽带速度满意,房间还算干净，离湖南路小吃街近。
i: 9, label: 1 5.1期间在这住的，位置还可以，在市委市政府附近，要去商业区和步行街得打车，屋里有蚊子，虽然空间挺大，晚上熄灯后把窗帘拉上简直是伸手不见五指，很适合睡觉，但是会被该死的蚊子吵醒！打死了两只，第二天早上还是发现又没打死的，卫生间挺大，但是设备很老旧。


## 加载模型

In [5]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')
print('token: ', token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

token:  BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 定义 collate_fn 方法

In [6]:
def collate_fn(data):
    sents =  [i[0] for i in data] # 猫 ...  16个  shape: [16, 500]
    labels = [i[1] for i in data] # 和 ...   16个

    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True
                                  )

    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    
    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [7]:

# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)


for i, values in enumerate(loader):
    input_ids, attention_mask, token_type_ids, labels = values
    # print(i, values)
    print(i, input_ids.shape, 
          attention_mask.shape, 
          token_type_ids.shape, 
          labels
         )
    if i >= 2:
        break
    

print(len(loader))

0 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1]
1 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0]
2 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0]
600


## 模型定义

In [8]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out

### 模型保存

In [ ]:
# 保存


model_save_path = 'chinese_class_mission_2023_4_10.pt'
# torch.save(model.state_dict(),  model_save_path)  # 推荐的文件后缀名是pt或pth

### 加载计算好的模型

In [9]:
# 加载保存的模型

model = Model()


model.load_state_dict(torch.load('chinese_class_mission_2023_4_10.pt'))


# 转移到 GPU 上计算
model = model.to(device)

In [10]:
# 封装检测类

def str_felling_detect(model, str_sents):
    

    out = token.encode_plus(str_sents,
                            truncation=True,# 当句子长度大于max_length时,截断
                            padding='max_length', # 一律补pad到 max_length长度
                            add_special_tokens=True,
                            max_length=500,
                            return_tensors=None,
                            return_token_type_ids=True,
                            return_attention_mask=True,
                            return_special_tokens_mask=True,
                            return_length=True
                           )
    
    input_ids = torch.tensor([out['input_ids']]).to(device)
    attention_mask = torch.tensor([out['attention_mask']]).to(device)
    token_type_ids = torch.tensor([out['token_type_ids']]).to(device)

    out_test = model(input_ids=input_ids,
                     attention_mask=attention_mask,                 
                     token_type_ids=token_type_ids
                    )
    
    print(out_test)
    out_res = out_test.argmax(dim=1).cpu().item()  # <class 'int'>

    if out_res == 1:
        print('1 正面情感输出')
    elif out_res == 0:
        print('0 负面情感输出')
    else:
        print('中性句子')

    return out_res

In [14]:

str_1 = '一直抄作业虽然短期来说能节省时间成本，但总归是要期末考试的'
str_2 = '你说得对，但是他们在1999年发明了世界上第一款gpu，并在今天用它培养出了chatgpt，而我们在2000年全面禁止了游戏机的研发，生产和销售，并在今天痛斥他们封锁我们'

str_felling_detect(model, str_2)

tensor([[0.9791, 0.0209]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0 负面情感输出


0

In [11]:
for i in range(10, 20):
    
    label = dataset[i][1]
    data_str = dataset[i][0]
    
    
    print('%s, label: %s data: %s' % (i, label,  data_str))
    
    str_felling_detect(model, data_str)
    print("")
    

10, label: 0 data: 我看过朋友的还可以，但是我订的书迟迟未到已有半个月，都没有收到打电话也没有用，以后你们订书一定要考虑好！当当实在是太慢了
tensor([[0.9973, 0.0027]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0 负面情感输出

11, label: 1 data: 还不错，设施稍微有点旧但是可以接收，但是606的价格还不含早餐有点高了。楼下的商场和超市很方便。下次来还会选择这家。
tensor([[0.0091, 0.9909]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
1 正面情感输出

12, label: 1 data: 轻便，方便携带，性能也不错，能满足平时的工作需要，对出差人员来说非常不错
tensor([[0.0174, 0.9826]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
1 正面情感输出

13, label: 1 data: 入住的是度假区的豪华海景房,前台给了5楼(最高6楼),然后差不多100%的海景,虽然是挂牌5星的,但是本人觉得是4星的标准,和我后来入住的5星喜来登差了蛮多的,不过整体来说还是符合他家的价钱的.
tensor([[0.1718, 0.8282]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
1 正面情感输出

14, label: 0 data: 1.模具有摄像头的位置，但是没有摄像头。 2.做工一般。 3.说明书和驱动都是电子版。 4.没有预装XP系统。
tensor([[0.9682, 0.0318]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0 负面情感输出

15, label: 0 data: 送的内胆包有点不好，还有外接电源中间连接处无法全部插入。续航时间也没有标称的那么长，希望京东能注意宣传的真实性。
tensor([[0.9531, 0.0469]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0 负面情感输出

16, label: 1 data: 这是我第1次给